In [12]:
from torch_compose.module import Nodule, NoduleGraph
import torch
from typing import Optional, Union, List, Dict
import graphlib
import networkx as nx

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
class MyNodule(Nodule):
    
    def __init__(self, input_keys, output_keys):
        super().__init__(input_keys, output_keys)
        
    def _forward(self, x):
        xx = x + 1
        return {'xx': xx}
    
    
class MyNodule2(Nodule):
    
    def __init__(self, input_keys, output_keys):
        super().__init__(input_keys, output_keys)
        
    def _forward(self, xx):
        return {'xxx': xx**2}

In [3]:
batch = {'x': torch.tensor(1.)}

In [4]:
nodule = MyNodule(input_keys=['x'], output_keys = ['xx'])
nodule2 = MyNodule2(input_keys=['xx'], output_keys = ['xxx'])

In [26]:
NoduleGraph(modules={'nodule': nodule, 'nodule2': nodule2})

NoduleGraph(
  (nodule2): MyNodule2()
)

In [13]:
nodules={'nodule': nodule, 'nodule2': nodule2}

In [22]:

graph = {}
nodule_map = {}
key_map = {}
keys = list(nodules.keys())

for ind, nodule in enumerate(nodules.values()):

    input_keys = nodule.input_keys
    output_keys = nodule.output_keys


    for key in output_keys:
        graph[key] = set(input_keys)
        key_map[tuple(output_keys)] = keys[ind]
    nodule_map[tuple(output_keys)] = nodule
    
graph = graph
sorter = graphlib.TopologicalSorter(graph)
ordering = [key for key in tuple(sorter.static_order())]


#

directed_graph = nx.DiGraph()
for output, inputs in graph.items():
    for input in inputs:
        directed_graph.add_edge(input, output)
directed_graph = directed_graph

ordered_submodules = []
ordered_keys = []
for key in ordering:
    for submodule_key in nodule_map.keys():
        if key in submodule_key:
            ordered_submodules.append(nodule_map.pop(submodule_key))
            ordered_keys.append(key_map.pop(submodule_key))
            break

In [9]:
test_nodule()

KeyError: 'c'